# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [68]:
import graphlab

In [69]:
import math

In [70]:
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [71]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [7]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [72]:
train_data,test_data = sales.random_split(.8,seed=0)

In [6]:
train_data['bedrooms_squared'] = train_data['bedrooms']*train_data['bedrooms']

In [7]:
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']

In [8]:
train_data['log_sqft_living']= train_data['sqft_living'].apply(lambda x: math.log(x))

In [9]:
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']

In [10]:
test_data['bedrooms_squared'] = test_data['bedrooms']*test_data['bedrooms']

In [11]:
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']

In [12]:
test_data['log_sqft_living']= test_data['sqft_living'].apply(lambda x: math.log(x))

In [13]:
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

In [40]:
round(test_data['bedrooms_squared'].mean(),2)

12.45

In [23]:
round(test_data[ 'bed_bath_rooms'].mean(),2)

7.5

In [24]:
round(test_data['log_sqft_living'].mean(),2)

7.55

In [25]:
round(test_data['lat_plus_long'].mean(),2)

-74.65

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [26]:
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = [ 'sqft_living', 'bedrooms', 'bathrooms', 'lat',  'long'], 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.040115     | 4074878.213096     | 236378.596455 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [31]:
model_1['coefficients']

name,index,value,stderr
(intercept),None,-56140675.7444,1649985.42028
sqft_living,None,310.263325778,3.18882960408
bedrooms,None,-59577.1160683,2487.27977322
bathrooms,None,13811.8405419,3593.54213297
lat,None,629865.789485,13120.7100323
long,None,-214790.285186,13284.2851607


In [33]:
model_1.evaluate(test_data)

{'max_error': 3950732.688377574, 'rmse': 231462.44070023226}

In [34]:
model_2.evaluate(test_data)

{'max_error': 3611744.7074838206, 'rmse': 230336.30198210376}

In [36]:
model_3.evaluate(test_data)

{'max_error': 6279035.883011501, 'rmse': 244024.98958661707}

In [28]:
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = [ 'sqft_living', 'bedrooms', 'bathrooms', 'lat',  'long',
                                                                                      'bed_bath_rooms'], 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.047123     | 4014170.932928     | 235190.935428 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [32]:
model_2['coefficients']

name,index,value,stderr
(intercept),None,-54410676.1152,1650405.16541
sqft_living,None,304.449298056,3.20217535637
bedrooms,None,-116366.04323,4805.54966546
bathrooms,None,-77972.3305131,7565.05991091
lat,None,625433.834953,13058.3530972
long,None,-203958.602959,13268.1283711
bed_bath_rooms,None,26961.6249091,1956.36561555


In [29]:
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = [ 'sqft_living', 'bedrooms', 'bathrooms', 'lat',  'long',
                                                                                      'bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living','lat_plus_long'], 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 9

Number of unpacked features : 9

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.019032     | 3193229.177908     | 228200.043155 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [6]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [ ]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [ ]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions

    # Then compute the residuals/errors

    # Then square and add them up

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [ ]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [ ]:
# create the remaining 3 features in both TEST and TRAIN data



* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [ ]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [ ]:
# Learn the three models: (don't forget to set validation_set = None)


In [ ]:
# Examine/extract each model's coefficients:


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [ ]:
# Compute the RSS on TRAINING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [ ]:
# Compute the RSS on TESTING data for each of the three models and record the values:


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [ ]:
GRADIENT DESCENT

In [76]:
def get_numpy_data(data_sframe, features, output):
    # add a constant column to an SFrame
    data_sframe['constant'] = 1 
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe = data_sframe.select_columns(features)
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray = data_sframe[output]
    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy() # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

In [44]:
def predict_outcome(feature_matrix, weights):
    predictions = np.dot(my_features, my_weights)
    return(predictions)

In [45]:
def feature_derivative(errors, feature):
    derivative = 2 * np.dot(feature,errors)
    return(derivative)

In [89]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    convergence = False
    weights = initial_weights
    transpose = np.transpose(feature_matrix)
    #CHECK FOR CONVERGENCE
    count = 0
    while(True):
        
        #CALCULATE the gradient
        predicted_output = np.dot(feature_matrix,weights)
        error = output - predicted_output        
        gradient = -2 * (np.dot(transpose,error))
        step = gradient * step_size
                         
        #UPDATE the weights i.e. parameter for next iteration THIS is not 
        weights = weights - step
        magnitude = np.sqrt(np.dot(gradient,gradient))
        print weights
        print magnitude
        count += 1
        #if count >100:
        #    break
        if magnitude > tolerance:
            continue
        else:
            break
    
    return (weights)
        
    

In [22]:
features_1 = [ 'sqft_living', 'bedrooms', 'bathrooms', 'lat',  'long']

In [ ]:
features_2 = [ 'sqft_living', 'bedrooms', 'bathrooms', 'lat',  'long', 'bed_bath_rooms']

In [23]:
features_3 = [ 'sqft_living', 'bedrooms', 'bathrooms', 'lat',  'long','bed_bath_rooms', 'bedrooms_squared', 'log_sqft_living','lat_plus_long'] 
                                                 

In [77]:
output = 'price'

In [104]:
tuple_1 = get_numpy_data(test_data, ['sqft_living'], output)

In [96]:
tuple_1[0]

array([[  1.00000000e+00,   1.18000000e+03,   1.34000000e+03],
       [  1.00000000e+00,   2.57000000e+03,   1.69000000e+03],
       [  1.00000000e+00,   7.70000000e+02,   2.72000000e+03],
       ..., 
       [  1.00000000e+00,   1.53000000e+03,   1.53000000e+03],
       [  1.00000000e+00,   1.60000000e+03,   1.41000000e+03],
       [  1.00000000e+00,   1.02000000e+03,   1.02000000e+03]])

In [33]:
tuple_1[1]

array([ 221900.,  538000.,  180000., ...,  360000.,  400000.,  325000.])

In [34]:
len(tuple_1[1])

17384

In [80]:
transpose = np.transpose(tuple_1[0])

In [81]:
pred_out = np.dot(tuple_1[0],np.array([-47000., 1.]))

In [82]:
error = tuple_1[1] - pred_out

In [83]:
grad = -2 * np.dot(transpose,error)

In [84]:
step =  grad * 7e-12 

In [85]:
np.array([-47000., 1.]) - step

array([-46999.85779866,    354.86068685])

In [86]:
np.sqrt(np.dot(grad,grad))

50551530774393.43

In [97]:
weights = regression_gradient_descent(tuple_1[0], tuple_1[1], np.array([-100000., 1., 1.]), 4e-12, 1e9)

[-99999.91164747    217.89658253    196.92903735]
7.30720205489e+13
[-99999.94015235    153.06856217    133.50564933]
2.26732209651e+13
[-99999.93238686    175.5618987     150.5856632 ]
7.0607945821e+12
[-99999.93584555    170.91513236    142.75832416]
2.27568239427e+12
[-99999.93579816    174.63083049    142.71624204]
928984105638.0
[-99999.93681401    175.69887247    140.31809353]
656307425178.0
[-99999.93747645    177.53542398    138.7079085 ]
610615351821.0
[-99999.93822543    179.08218133    136.90918121]
593078765307.0
[-99999.93892505    180.66860703    135.2234583 ]
578705920128.0
[-99999.93961798    182.19370677    133.55591875]
564945676163.0
[-99999.94029152    183.68996794    131.9347091 ]
551538681425.0
[-99999.94095012    185.14844983    130.34986642]
538452422879.0
[-99999.94159289    186.57303406    128.803277  ]
525676912708.0
[-99999.94222059    187.96359899    127.29318056]
513204543690.0
[-99999.94283347    189.32123894    125.81897573]
501028100319.0
[-99999.943431

In [91]:
weights

array([-46999.88716555,    281.91211912])

In [98]:
weights

array([ -9.99999688e+04,   2.45072603e+02,   6.52795277e+01])

In [93]:
(1430*281.91211912)-46999.88716555

356134.44317605

In [100]:
(1430*2.45072603e+02)+(1780*6.52795277e+01)-9.99999688e+04

366651.412796

In [99]:
test_data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901,1.5,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000,2,0
1202000200,2014-11-03 00:00:00+00:00,233000.0,3.0,2.0,1710.0,4697,1.5,0
8562750320,2014-11-10 00:00:00+00:00,580500.0,3.0,2.5,2320.0,3980,2,0
7589200193,2014-11-10 00:00:00+00:00,535000.0,3.0,1.0,1090.0,3000,1.5,0
2078500320,2014-06-20 00:00:00+00:00,605000.0,4.0,2.5,2620.0,7553,2,0
7766200013,2014-08-11 00:00:00+00:00,775000.0,4.0,2.25,4220.0,24186,1,0
9478500640,2014-08-19 00:00:00+00:00,292500.0,4.0,2.5,2250.0,4495,2,0
9558200045,2014-08-28 00:00:00+00:00,289000.0,3.0,1.75,1260.0,8400,1,0
8820901275,2014-06-10 00:00:00+00:00,571000.0,4.0,2.0,2750.0,7807,1.5,0


In [103]:
#for 2nd model
x = tuple_1[1]-np.dot(tuple_1[0],np.array([ -9.99999688e+04,   2.45072603e+02,   6.52795277e+01]))
np.dot(x,x)

270263446455930.0

In [105]:
# for 1st model
x = tuple_1[1]-np.dot(tuple_1[0],np.array([-46999.88716555,    281.91211912]))
np.dot(x,x)

275400047593086.31